In [12]:
import pandas as pd

In [13]:
data = {
    "transaction_id": [1, 2, 3, 4],
    "user_id": ["U1", "U1", "U2", "U1"],
    "merchant": ["M1", "M2", "M1", "M1"],
    "transaction_time": [
        "2025-09-16 14:32:10",
        "2025-09-16 16:10:21",
        "2025-09-16 02:45:35",
        "2025-09-17 09:12:22",
    ],
    "amount": [100.5, 200.0, 500.0, 50.7],
}
df = pd.DataFrame(data)
df["transaction_time"] = pd.to_datetime(df["transaction_time"])

In [14]:
agg_features = df.groupby("user_id")["amount"].agg(
    user_transaction_count = "count",
    user_total_amount = "sum",
    user_mean_amount = "mean",
    user_std_amount = "std"
).reset_index()
df = df.merge(agg_features, on="user_id", how="left")

In [15]:
df["amount_to_mean_ratio"] = df["amount"] / df["user_mean_amount"]

merchant_counts = df.groupby("user_id")["merchant"].nunique().reset_index()
merchant_counts.rename(columns={"merchant": "unique_merchant"},inplace=True)
df = df.merge(merchant_counts, on="user_id", how="left")

In [21]:
df = df.sort_values(["user_id", "transaction_time"])

df["rolling_meaning_amount"] = df.groupby("user_id")["amount"].transform(
    lambda x: x.rolling(window =2, min_periods=1).mean()
)

In [22]:
df

,transaction_id,user_id,merchant,transaction_time,amount,user_transaction_count,user_total_amount,user_mean_amount,user_std_amount,amount_to_mean_ratio,unique_merchant,rolling_meaning_amount
0,1,U1,M1,2025-09-16 14:32:10,100.5,3,351.2,117.066667,76.016204,0.858485,2,100.50
1,2,U1,M2,2025-09-16 16:10:21,200.0,3,351.2,117.066667,76.016204,1.708428,2,150.25
3,4,U1,M1,2025-09-17 09:12:22,50.7,3,351.2,117.066667,76.016204,0.433087,2,125.35
2,3,U2,M1,2025-09-16 02:45:35,500.0,1,500.0,500.000000,NaN,1.000000,1,500.00
